In [21]:
from config.es import es
from config.settings import settings
from models.model import AnnoqData
from itertools import islice

In [46]:
async def get_annotations():
    resp = await es.search(
          index=settings.ES_INDEX,
          query={"match_all": {}},
          size=20,
    )

    results = [AnnoqData(id=hit['_id'], 
                         chr = hit['_source']['chr'],
                         pos = hit['_source']['pos'],
                         ref = hit['_source']['ref'],
                         alt = hit['_source']['alt'],
                         ANNOVAR_ensembl_Effect = hit['_source']['ANNOVAR_ensembl_Effect'],
                         ANNOVAR_ensembl_Transcript_ID = hit['_source'].get('ANNOVAR_ensembl_Transcript_ID', None),
                         ANNOVAR_ensembl_Gene_ID = hit['_source'].get('ANNOVAR_ensembl_Gene_ID', None),
                         ANNOVAR_ensembl_summary = hit['_source']['ANNOVAR_ensembl_summary'],
                         SnpEff_ensembl_Effect = hit['_source']['SnpEff_ensembl_Effect'],
                         SnpEff_ensembl_Effect_impact = hit['_source']['SnpEff_ensembl_Effect_impact']) 
              for hit in resp['hits']['hits']]
        
    return results

results = await get_annotations()
results

[AnnoqData(id='2:10662G>C', chr='2', pos=10662, ref='G', alt='C', ANNOVAR_ensembl_Effect='intergenic', ANNOVAR_ensembl_Transcript_ID=None, ANNOVAR_ensembl_Gene_ID=None, ANNOVAR_ensembl_summary='(0):intergenic(1)', SnpEff_ensembl_Effect='intergenic_region', SnpEff_ensembl_Effect_impact='MODIFIER'),
 AnnoqData(id='6:89961G>A', chr='6', pos=89961, ref='G', alt='A', ANNOVAR_ensembl_Effect='intergenic', ANNOVAR_ensembl_Transcript_ID=None, ANNOVAR_ensembl_Gene_ID=None, ANNOVAR_ensembl_summary='(0):intergenic(1)', SnpEff_ensembl_Effect='intergenic_region', SnpEff_ensembl_Effect_impact='MODIFIER'),
 AnnoqData(id='6:147651A>C', chr='6', pos=147651, ref='A', alt='C', ANNOVAR_ensembl_Effect='upstream', ANNOVAR_ensembl_Transcript_ID='ENST00000406017|ENST00000436899', ANNOVAR_ensembl_Gene_ID='ENSG00000217929|ENSG00000170590', ANNOVAR_ensembl_summary='ENSG00000170590(1):upstream(1)|ENSG00000217929(1):upstream(1)', SnpEff_ensembl_Effect='intergenic_region|upstream_gene_variant', SnpEff_ensembl_Effect

In [41]:
async def search_by_ID(id:str):

    response = await es.get(
        index = settings.ES_INDEX,
        id = id
    )

    results = AnnoqData(id=response['_id'], **dict(islice(response['_source'].items(), 10)))

    return results

results = await search_by_ID("1:13380C>G")
results

AnnoqData(id='1:13380C>G', chr='1', pos=13380, ref='C', alt='G', ANNOVAR_ensembl_Effect='ncRNA_exonic|ncRNA_intronic|downstream', ANNOVAR_ensembl_Transcript_ID='ENST00000456328|ENST00000515242|ENST00000450305|ENST00000518655|ENST00000423562|ENST00000438504|ENST00000488147|ENST00000538476|ENST00000541675', ANNOVAR_ensembl_Gene_ID='ENSG00000223972|ENSG00000227232', ANNOVAR_ensembl_summary='ENSG00000223972(4):ncRNA_exonic(2)ncRNA_intronic(2)|ENSG00000227232(5):downstream(5)', SnpEff_ensembl_Effect='downstream_gene_variant|splice_region_variant&intron_variant|non_coding_transcript_exon_variant|intron_variant', SnpEff_ensembl_Effect_impact='MODIFIER|LOW')